# 24. GRADUATE ADMISSION: CLASSIFICATION MODELS
---

## 1. Introducing the Data
To turn the dataset into classification, we want to convert the target column into classes:
- `1=admit` (chance of admit is above 0.85)
- `2=put on waitlist` (chance of admit is above 0.70 but below 0.85)
- `3=do not admit` (chance of admit is below 0.70)

In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", 99)
pd.set_option("display.max_rows", 999)
pd.set_option('precision', 3)

admission = pd.read_csv('data/Admission_1.1.csv')
admission['LOR'] = admission['LOR ']
admission['Chance of Admit'] = admission['Chance of Admit ']
admission_1 = admission.drop(['Serial No.', 'LOR ', 'Chance of Admit '], axis=1)
print(admission_1.shape)
print(admission_1.columns)
admission_1.head()

(500, 8)
Index(['GRE Score', 'TOEFL Score', 'University Rating', 'SOP', 'CGPA',
       'Research', 'LOR', 'Chance of Admit'],
      dtype='object')


,GRE Score,TOEFL Score,University Rating,SOP,CGPA,Research,LOR,Chance of Admit
0,337,118,4,4.5,9.65,1,4.5,0.92
1,324,107,4,4.0,8.87,1,4.5,0.76
2,316,104,3,3.0,8.00,1,3.5,0.72
3,322,110,3,3.5,8.67,1,2.5,0.80
4,314,103,2,2.0,8.21,0,3.0,0.65


## 2. Modifying the Target for Classification

In [16]:
admission_2 = admission_1.copy()
y = admission_2['Chance of Admit']
for i in y:
    if i>=0.85:
        y = y.replace(i,1)
    elif i<=0.70:
        y = y.replace(i,3)
    else: y = y.replace(i,2)

y = y.astype('int')
y.value_counts()

3    213
2    178
1    109
Name: Chance of Admit, dtype: int64

In [18]:
admission_2['Chance of Admit'] = y
admission_2.head()

,GRE Score,TOEFL Score,University Rating,SOP,CGPA,Research,LOR,Chance of Admit
0,337,118,4,4.5,9.65,1,4.5,1
1,324,107,4,4.0,8.87,1,4.5,2
2,316,104,3,3.0,8.00,1,3.5,2
3,322,110,3,3.5,8.67,1,2.5,2
4,314,103,2,2.0,8.21,0,3.0,3


## 3. Train-Test Splitting

In [21]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(admission_2, test_size=0.2, random_state=42)
X_train = train.drop('Chance of Admit', axis=1)
y_train = train['Chance of Admit']
X_test = test.drop('Chance of Admit', axis=1)
y_test = test['Chance of Admit']

print('X_Train:', X_train.shape, '\n', 'Train:', train.shape)

X_Train: (400, 7) 
 Train: (400, 8)


## 4. Scaling the Data

In [26]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train[:5]

array([[0.62      , 0.67857143, 0.5       , 0.625     , 0.65064103,
        1.        , 0.71428571],
       [0.52      , 0.67857143, 0.75      , 0.75      , 0.55769231,
        0.        , 1.        ],
       [0.26      , 0.35714286, 0.5       , 0.625     , 0.54487179,
        0.        , 0.42857143],
       [0.48      , 0.53571429, 0.25      , 0.375     , 0.47115385,
        0.        , 0.71428571],
       [0.36      , 0.5       , 0.5       , 0.625     , 0.45192308,
        1.        , 0.28571429]])

## 5. Training a Random Forest Classifier